# Parte 1

## Imports

- **numpy**: 
- **pandas**: 
- **nltk**: Para POS tagging, tokenização de sentenças e demais itens inerentes a NLP
- **emoji**: Para converter emojis no texto para suas respectivas descriçrões
- _emot_: Considerar o uso para extrair emojis e emoticons do texto
- **re**: Para encontrar padrões por meio de expressões regulares

In [18]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import mac_morpho
import emoji
import re

## Leitura dos dados

Lendo os dados do arquivo **banzeiro.json** em duas listas de objetos, que serão representados por `dicts`. Uma lista guardará os dados com os emojis substituídos pelas suas respectivas descrições e a outra, o conteúdo original.

In [19]:
posts = pd.read_json("banzeiro.json")
posts.head()

,key,datetime,img_urls,caption,comments
0,https://www.instagram.com/p/B2mMB_OHbfI/,2019-09-19 14:47:48+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,https://www.instagram.com/p/B2ef0osHDik/,2019-09-16 15:06:49+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,https://www.instagram.com/p/B2ZXEGSn1di/,2019-09-14 15:14:05+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,https://www.instagram.com/p/B2UpaJ5HLWg/,2019-09-12 19:23:34+00:00,[],"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,https://www.instagram.com/p/B2PCM4Qn9mD/,2019-09-10 14:59:22+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."


Verificando a estrutura dos dados, é possível perceber que existem colunas que não contém texto excrito por usuários, que uma publicação não possui legenda (`caption`) e que nem todas as instâncias possuem um vetor de comentários a ser processado (`comments`).

In [20]:
posts.count()

key         778
datetime    778
img_urls    778
caption     777
comments    606
dtype: int64

# Parte 2

## Limpeza de dados

Algumas operações de limpeza dos dados deverão ser realizadas, considerando os seguintes pontos:

- Algumas colunas não são relevantes no processamento de texto
- Comentários que apenas mencionam outros usuários não expressam uma opinião
- Publicações sem legenda não deixam claro a que se endereça a opinião
- Publicações sem comentários não possuem opiniões a serem mineradas


### Removendo colunas sem importância

In [21]:
posts = posts.drop(["key", "datetime", "img_urls"], axis=1)

In [22]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."
774,"Pôr do sol em São Gabriel da Cachoeira, por Fe...",NaN
775,Viaje com os sabores e combinações exóticas do...,NaN
776,Petit gateau amazônico.\nVenha saborear essa m...,"[{'author': 'victorer83', 'comment': '@marinay..."


### Removendo linhas onde caption ou comments são nulos

In [23]:
posts = posts.dropna()

In [24]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
765,"Desbrave novos sabores, embale seu paladar e d...","[{'author': 'mellothz', 'comment': 'Estive no ..."
767,Pastel de pato. Deliciosamente incomparável. U...,"[{'author': 'chriscalderaro', 'comment': 'Eu a..."
771,O chef Felipe Schaedler foi convidado para par...,"[{'author': 'thiagosuico', 'comment': '@felipe..."
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."


### Removendo comentários que somente mencionam outros usuários

Fazendo o uso de Expressão Regular, qualquer comentário que contenha apenas nomes de outros usuários, com objetivo único de mencioná-los na publicação, será removido. Linhas que eventualmente fiquem sem comentários serão descartadas.

In [25]:
for index, row in posts.iterrows():
    for comment in row["comments"]:
        if re.search("^(@[a-z0-9_.]*\s*)+$", comment["comment"]):
            posts.at[index, "comments"].remove(comment)
posts = posts.dropna()

In [26]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
765,"Desbrave novos sabores, embale seu paladar e d...","[{'author': 'mellothz', 'comment': 'Estive no ..."
767,Pastel de pato. Deliciosamente incomparável. U...,"[{'author': 'chriscalderaro', 'comment': 'Eu a..."
771,O chef Felipe Schaedler foi convidado para par...,"[{'author': 'thiagosuico', 'comment': '@felipe..."
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."


# Parte 3

## Convertendo o dataframe para dict

Com a parte de limpeza dos dados finalizada e antes do processamento realizado pelo NLTK, os dados serão representados em forma de **dict**.

In [27]:
posts = posts.to_dict('records')


## Tokenizando sentenças e Strings

Fazendo o uso do **nltk**, serão tokenizadas as sentenças e palavras do dataset. As sentenças serão quebradas com o tokenizador **Punkt** treinado para o Português, e as palavras, quebrando nos espaços em branco.

In [28]:
nltk.download("punkt")
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

[nltk_data] Downloading package punkt to /home/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### POS Tagging

A marcação de _Part-of-Speech_ será feita com _taggers_ do NLTK treinados com o _corpus_ **Mac Morpho** de notícias do jornal Folha de S. Paulo de 1994. Para manter o código mais limpo, a tokenização e as marcações ocorrerão em conjunto.

In [29]:
tsents = mac_morpho.tagged_sents()

- *noun_tagger* é o tagger default, se nenhum outro resolve, ele assume que é um substantivo
- *unigram_tagger* é um tagger para unigramas, baseado na maior frequência de tags encontrada nos corpus de treino
- *bigram_tagger* é um tagger para bigramas que avalia a palavra levando em consideração a palavra anterior, afim de resolver ambiguidades

In [30]:
noun_tagger = nltk.DefaultTagger('N')
unigram_tagger = nltk.UnigramTagger(tsents, backoff=noun_tagger)
bigram_tagger = nltk.BigramTagger(tsents, backoff=unigram_tagger)

In [31]:
for p in posts:
    sentences = sent_tokenizer.tokenize(p["caption"])
    print("Caption: " )
    p["caption"] = [bigram_tagger.tag(nltk.word_tokenize(sentence)) for sentence in sentences]
    print(p["caption"], "\n\n")
    for c in p["comments"]:
        print("Comment: " )
        sentences = sent_tokenizer.tokenize(c["comment"])
        c["comment"] = [bigram_tagger.tag(nltk.word_tokenize(sentence)) for sentence in sentences]
        print(c["comment"], "\n\n")

Caption: 
[[('Já', 'ADV'), ('provou', 'V'), ('os', 'ART'), ('nossos', 'PROADJ'), ('Chás', 'N'), ('Gelados', 'N'), ('?', '?')], [('As', 'ART'), ('três', 'NUM'), ('combinações', 'N'), ('que', 'PRO-KS-REL'), ('vem', 'V'), ('chamando', 'V'), ('a', 'ART'), ('atenção', 'N'), ('dos', 'N'), ('nossos', 'PROADJ'), ('clientes', 'N'), ('que', 'PRO-KS-REL'), ('optam', 'N'), ('por', 'PREP'), ('uma', 'ART'), ('opção', 'N'), ('refrescante', 'N'), ('e', 'KC'), ('não', 'ADV'), ('alcoólica', 'ADJ'), (':', ':'), ('Hibisco', 'N'), ('com', 'PREP'), ('Abacaxi', 'N'), ('de', 'PREP'), ('Manaus', 'NPROP'), (',', ','), ('Limão', 'NPROP'), ('com', 'PREP'), ('Alecrim', 'N'), ('e', 'KC'), ('Maracujá', 'N'), ('com', 'PREP'), ('Cumaru', 'N'), ('.', '.')]] 


Comment: 
[[('Deliciosos', 'N')]] 


Caption: 
[[('Uma', 'ART'), ('das', 'NPROP'), ('entradas', 'N'), ('exclusivas', 'ADJ'), ('do', 'KS'), ('Banzeiro', 'N'), ('Manaus', 'NPROP'), ('.', 'NPROP')], [('Experimente', 'V|+'), ('o', 'PROPESS'), ('nosso', 'PROADJ'), ('T

[[('Peixe', 'N'), ('combina', 'V'), ('c', 'N'), ('suco', 'N'), ('doce', 'ADJ'), ('mel', 'N'), ('!', '!')], [('!', '!')]] 


Caption: 
[[('Hoje', 'ADV'), ('não', 'ADV'), ('tem', 'V'), ('desculpa', 'N'), ('!', '!')], [('É', 'V'), ('dia', 'N'), ('de', 'PREP'), ('experimentar', 'V'), ('o', 'ART'), ('nosso', 'PROADJ'), ('delicioso', 'ADJ'), ('Parmegiana', 'N'), ('de', 'PREP'), ('Pirarucu', 'N'), ('.', '.')], [(':', ':'), (')', ')')]] 


Comment: 
[[('Deve', 'VAUX'), ('ser', 'VAUX'), ('delicioso', 'ADJ'), ('!', '!'), ('!', '!')], [('!', '!')]] 


Comment: 
[[('@', 'N'), ('lanadelour', 'N'), ('hahahhahahahahahahahhahahaha', 'N'), ('vamo', 'N'), ('.', '.')], [('Desisti', 'N'), ('de', 'PREP'), ('sabadin', 'N'), ('!', '!')]] 


Caption: 
[[('Começando', 'V'), ('a', 'ART'), ('semana', 'N'), ('com', 'PREP'), ('um', 'ART'), ('de', 'PREP|+'), ('nossos', 'PROADJ'), ('clássicos', 'N'), (':', ':'), ('Pirarucu', 'N'), ('à', 'PREP|+'), ('Belle', 'NPROP'), ('Meunière', 'N'), ('!', '!')]] 


Comment: 
[[('

[[('😉😉😉', 'N')]] 


Caption: 
[[('#', 'N'), ('repost', 'N'), ('Amazonas', 'NPROP'), (',', ','), ('com', 'PREP'), ('muito', 'PROADJ'), ('respeito', 'N'), ('eu', 'PROPESS'), ('fiz', 'V'), ('o', 'ART'), ('meu', 'PROADJ'), ('melhor', 'ADJ'), ('aqui', 'ADV'), ('hoje', 'ADV'), ('.', '.')], [('Espero', 'V'), ('que', 'KS'), ('todo', 'PROADJ'), ('esse', 'PROADJ'), ('trabalho', 'N'), ('reflita', 'V'), ('em', 'PREP|+'), ('coisas', 'N'), ('boas', 'ADJ'), ('para', 'PREP'), ('Manaus', 'NPROP'), ('e', 'KC'), ('o', 'ART'), ('estado', 'N'), ('todo', 'PROADJ'), ('.', '.')], [('Muito', 'ADV'), ('obrigado', 'PCP'), ('a', 'PREP|+'), ('todos', 'PROADJ'), ('os', 'ART'), ('envolvidos', 'N'), ('.', '.')], [('Hoje', 'ADV'), ('eu', 'PROPESS'), ('realizei', 'V'), ('algo', 'PROSUB'), ('muito', 'ADV'), ('maior', 'ADJ'), ('que', 'PRO-KS-REL'), ('um', 'ART'), ('sonho', 'N'), ('aqui', 'ADV'), ('no', 'PREP|+'), ('@', 'N'), ('ssgastronomika', 'N'), ('🙏🏼', 'N'), ('#', 'N'), ('amazoniainvisivel', 'N')]] 


Comment: 
[[('P

[[('Olha', 'V'), ('ai', 'IN'), ('@', 'N'), ('neivaaraujo', 'N')]] 


Comment: 
[[('Boa', 'ADJ'), ('Noite', 'NPROP'), ('por', 'PREP'), ('gentileza', 'N'), ('poderia', 'VAUX'), ('me', 'PROPESS'), ('responder', 'V'), ('se', 'KS'), ('vão', 'VAUX'), ('funcionar', 'V'), ('no', 'PREP|+'), ('feriado', 'N'), ('sexta', 'N'), ('dia', 'N'), ('20/11', 'N'), ('@', 'N'), ('restaurantebanzeiro', 'N'), ('fico', 'V'), ('no', 'PREP|+'), ('aguardo', 'N'), ('agradeço', 'N'), ('desde', 'PREP'), ('já', 'ADV'), ('😉😘', 'N')]] 


Caption: 
[[('Que', 'PROADJ'), ('tal', 'PROSUB'), ('experimentar', 'V'), ('uma', 'ART'), ('de', 'PREP|+'), ('nossas', 'PROADJ'), ('delícias', 'N'), ('regionais', 'ADJ'), ('?', '?')], [('#', 'N'), ('Banzeiro', 'N'), ('#', 'N'), ('Food', 'NPROP'), ('#', 'N'), ('InstaFood', 'N'), ('#', 'N'), ('Manaus', 'NPROP')]] 


Comment: 
[[('Amigos', 'N'), ('que', 'PRO-KS-REL'), ('prato', 'N'), ('é', 'V'), ('esse', 'PROADJ'), ('?', '?')], [('@', 'N'), ('restaurantebanzeiro', 'N')]] 


Comment: 
[[('O